In [1]:
import tensorflow as tf
import numpy as np

In [2]:
t = 5
for v in range(t):
    with open("GFSC/reuters_views/view" + str(v) + ".npy", 'rb') as fi:
        tmp = tf.cast([np.load(fi)], dtype=tf.float32)
        if v == 0:
            X = tf.linalg.matmul(tmp, tmp, transpose_b=True)
        else:
            X = tf.concat([X, tf.linalg.matmul(tmp, tmp, transpose_b=True)], axis=0)
print(X.shape)

(5, 1200, 1200)


In [3]:
k = 6
steps = 1
alphas, betas, gammas = [1.0], [1000.0], [0.01]
n = X.shape[2]

In [4]:
@tf.function
def fusion():
    s = tf.where(S > 0, S, tf.zeros(S.shape))
    s = tf.math.divide(s + tf.transpose(s), 2.0)
    z_sum = tf.zeros(s.shape)
    for i in range(t):
        z = tf.linalg.matmul(
                tf.linalg.inv(X[i] + i_alpha + i_beta * wv[i]),
                s * beta * wv[i] + X[i])
        z = tf.where(z[i] > 0, z, tf.zeros(z.shape))
        z = tf.math.divide(z + tf.transpose(z), 2.0)
        wv[i].assign(0.5 / tf.norm(z - s))
        z = tf.math.multiply(z, wv[i])
        z_sum = tf.math.add(z_sum, z)

    l = tf.math.subtract(tf.linalg.diag(tf.math.reduce_sum(s, 0)), s)
    (_, f) = tf.linalg.eigh(l)
    f = tf.slice(f, [0, 0], [f.shape[0], k])
    p = tf.norm(tf.math.subtract(f[0][:], f), axis=1, keepdims=True)
    for i in range(1, n):
         p = tf.concat([p, tf.norm(tf.math.subtract(f[0][:], f), axis=1, keepdims=True)], 1)
    p = tf.math.multiply(tf.math.square(p), reg)
    s_old = s
    s = tf.math.divide((z_sum - p), tf.math.reduce_sum(wv, 0))
    return s, tf.math.divide(tf.norm(tf.math.subtract(s, s_old)), tf.norm(s_old)), f

In [5]:
for alpha in alphas:
    for beta in betas:
        for gamma in gammas:
            reg = gamma / (4.0 * beta)
            wv = tf.Variable([1.0 / t for _ in range(t)], dtype=tf.float32)
            i_alpha = tf.math.multiply(tf.eye(n, dtype=tf.float32), alpha)
            i_beta = tf.math.multiply(tf.eye(n, dtype=tf.float32), beta)
            S = tf.eye(n, dtype=tf.float32)
            for step in range(steps):
                (S, delta, F) = fusion()
                print('Num steps:', step)
                if step > 5 and delta < 1.e-3:
                    break
            print(S)
            with open("fusion_matrix/S_reuters.npy", 'wb') as fi:
                np.save(fi, S.numpy())

Num steps: 0
tf.Tensor(
[[ 9.83875930e-01 -1.38772011e-05 -1.73127337e-05 ...  1.07561555e-05
  -8.08928053e-06  4.98790723e-06]
 [-1.46818184e-05  9.93063629e-01  3.92601396e-05 ... -1.62300580e-06
   2.53536405e-06  1.80381005e-06]
 [-1.81173509e-05  3.92601396e-05  9.83804107e-01 ...  3.89044817e-06
  -2.52861071e-07 -2.69203019e-06]
 ...
 [ 1.03538468e-05 -1.22069741e-06  4.29275678e-06 ...  9.92831469e-01
   9.46421733e-06 -1.63249604e-06]
 [-8.49158914e-06  2.93767243e-06  1.49447274e-07 ...  9.46421733e-06
   9.92783904e-01  3.15799775e-06]
 [ 4.58559907e-06  2.20611855e-06 -2.28972158e-06 ... -1.63249604e-06
   3.15799775e-06  9.95158911e-01]], shape=(1200, 1200), dtype=float32)


In [6]:
# x = tf.constant([[2., 16., 128.],
#                  [4., 32., 256.],
#                  [8., 64., 512.]], dtype=tf.float32)
# #
# # # print(tf.slice(x, [0, 0], [x.shape[0], 3]))
# # y = tf.cast([1], dtype=tf.float32)
# y = 0.5 / tf.norm(x)
# print(y)
# # tf.concat([y, y], axis=0)
# print(tf.math.multiply(x, y))
